# Basic Agent with langchain


* Conversational Agent with Context Awareness
* Question Answering Agent
* Data Analysis Agent

In [1]:
# %pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

Note: you may need to restart the kernel to use updated packages.


## Conversational Agent with Context Awareness

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Now get the API key (it should be loaded from .env file)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
else:
    print("Warning: OPENAI_API_KEY not found in environment variables.")
    print("Please make sure you have a .env file with OPENAI_API_KEY=your_api_key")

### Initialize language model

In [15]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### in-memory store for chat histories

In [4]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

### Prompt template

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

### Combine the prompt and model into a runnable chain

In [6]:
chain = prompt | llm

### Wrap the chain with message history

In [7]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

### Example usage


In [8]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
AI: Your previous message was, "Hello! How are you?" How can I assist you further?


### Print the conversation history

In [9]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: What was my previous message?
ai: Your previous message was, "Hello! How are you?" How can I assist you further?


## Question Answering Agent

We will use the same model and but prompt template inste of ChatPromptTemplate.

* For completion-style LLMs. we can use prompt template with placeholders, and fill them in at runtime.

* For chat-based LLMs, we use ChatPromptTemplate with messages. Consists of a sequence of messages (SystemMessage, HumanMessage, AIMessage, etc.)

In [10]:
from langchain.prompts import PromptTemplate

In [11]:
template = """
You are a helpful AI assistant. Your task is to answer the user's question to the best of your ability.

User's question: {question}

Please provide a clear and concise answer:
"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [12]:
qa_chain = prompt | llm

##### Define the get_answer function

In [13]:
def get_answer(question):
    """
    Get an answer to the given question using the QA chain.
    """
    input_variables = {"question": question}
    response = qa_chain.invoke(input_variables).content
    return response

##### Example usage

In [14]:
question = "What is the capital of France?"
answer = get_answer(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What is the capital of France?
Answer: The capital of France is Paris.


## Data Analysis Agent

In [ ]:
# %pip install tabulate

In [16]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set a random seed for reproducibility
np.random.seed(42)

In [6]:
# # Generate sample data
# n_rows = 1000

# # Generate dates
# start_date = datetime(2022, 1, 1)
# dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# # Define data categories
# makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
# models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
# colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# # Create the dataset
# data = {
#     'Date': dates,
#     'Make': np.random.choice(makes, n_rows),
#     'Model': np.random.choice(models, n_rows),
#     'Color': np.random.choice(colors, n_rows),
#     'Year': np.random.randint(2015, 2023, n_rows),
#     'Price': np.random.uniform(20000, 80000, n_rows).round(2),
#     'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
#     'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
#     'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
#     'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
# }

# # Create DataFrame and sort by date
# df = pd.DataFrame(data).sort_values('Date')

# # Save to CSV file
# csv_filename = "car_sales_data.csv"
# df.to_csv(csv_filename, index=False)
# print(f"Data saved to {csv_filename}")

# # Display sample data and statistics
# print("\nFirst few rows of the generated data:")
# print(df.head())

# print("\nDataFrame info:")
# df.info()

# print("\nSummary statistics:")
# print(df.describe())

### Load data from CSV file

In [17]:
df = pd.read_csv('car_sales_data.csv')

if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])

print(f"DataFrame shape: {df.shape}")

print("\nFirst few rows of loaded data:")
print(df.head())

print("\nData types:")
print(df.dtypes)

DataFrame shape: (1000, 10)

First few rows of loaded data:
        Date      Make      Model   Color  Year     Price  Mileage  \
0 2022-01-01  Mercedes      Sedan    Blue  2020  43639.25  65879.0   
1 2022-01-02    Toyota  Hatchback    Gray  2016  44983.53  62313.0   
2 2022-01-03   Hyundai        Van     Red  2019  47054.44  49459.0   
3 2022-01-04    Toyota      Sedan    Blue  2020  69614.21  21126.0   
4 2022-01-05    Nissan      Coupe  Silver  2018  25880.32  38198.0   

   EngineSize  FuelEfficiency SalesPerson  
0         1.6            26.6       David  
1         3.5            23.4         Eva  
2         2.5            29.3         Bob  
3         3.0            22.0         Bob  
4         4.0            26.7         Bob  

Data types:
Date              datetime64[ns]
Make                      object
Model                     object
Color                     object
Year                       int64
Price                    float64
Mileage                  float64
EngineSize 

##### Create Data Analysis Agent

In [20]:


agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
print("Data Analysis Agent is ready. You can now ask questions about the data.")

Data Analysis Agent is ready. You can now ask questions about the data.


In [21]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

Example usage

In [22]:
ask_agent("What are the column names in this dataset?")
ask_agent("How many rows are in this dataset?")
ask_agent("What is the average price of cars sold?")

C:\Users\Tiu User\AppData\Local\Temp\ipykernel_41756\610968568.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run({




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns.tolist()'}`


['Date', 'Make', 'Model', 'Color', 'Year', 'Price', 'Mileage', 'EngineSize', 'FuelEfficiency', 'SalesPerson']The column names in the dataset are:

1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson

> Finished chain.
Question: What are the column names in this dataset?
Answer: The column names in the dataset are:

1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson
---


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


1000The dataset contains 1,000 rows.

> Finished chain.
Question: How many rows are in this dataset?
Answer: The dataset contains 1,000 rows.
---


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "average_price = df['Price'].mean()\naverage_price"}`


50557.66065999